In [2]:
import math
import numpy as np
from qiskit import *
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator
from qiskit import IBMQ
import matplotlib
%matplotlib inline

IBMQ.load_account()

C:\Users\Sorin\Anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:181: UserWarning: Credentials are already in use. The existing account in the session will be replaced.
  warnings.warn('Credentials are already in use. The existing '


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
def loadQuantumImage(path):
    from PIL import Image
    image= Image.open(path)
    (width, height) = image.size
    pixels = image.load()

    ssum = 0
    for i in range(width):
        for j in range(height):
            ssum += pixels[i,j]**2

    m = math.floor(math.log(height, 2))
    n = math.floor(math.log(width, 2))
    stateVector = np.zeros(2**(m+n))
    for i in range(width):
        for j in range(height):
            stateVector[i*height + j] = pixels[i,j]/math.sqrt(ssum)
            
    return stateVector, m, n

In [4]:
def exportQuantumImage(counts, shots, height, width):
    from PIL import Image

    r = math.floor(math.log(width, 2))
    im= Image.new("L", (width, height))
    pixels = im.load()

    maxAmplitude = 256
    medAmplitude = maxAmplitude/2
    med = shots/(height * width)
    for key in counts:
        i = int(key[0:r], 2)
        j = int(key[r:], 2)
        val = round((((counts[key] - med)/med)*medAmplitude) + medAmplitude)
        pixels[i,j] = (val)
        
    return im

In [5]:
def Increment(increment, quantumRegister, circuit):
    from qiskit.aqua.components.qfts import Standard as qft
    from qiskit.aqua.components.iqfts import Standard as iqft
    
    n = len(quantumRegister)
    
    qft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)
    
    for j in range(n):
        circuit.u1((np.pi*increment)/(2**(n-1-j)), quantumRegister[j])
        
    iqft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)
    

In [6]:
def Rotate(rowReg, columnReg, circuit):
    circuit.x(rowReg)
    circuit.x(columnReg)

In [7]:
def Transpose(rowReg, columnReg, circuit):
    m = rowReg.size
    n = columnReg.size
    
    if (n != m):
        raise "Can only transpose square images" 
    
    for i in range(n):
        circuit.swap(rowReg[i], columnReg[i])

In [ ]:
stateVector, m, n = loadQuantumImage('lena64.png')

state = [complex(x) for x in stateVector]
indexes = range(m+n+1)

columnReg = QuantumRegister(n)
rowReg = QuantumRegister(m)
ancilaReg = QuantumRegister(1)
creg = ClassicalRegister(n+m+1)

circ = QuantumCircuit(ancilaReg, rowReg, columnReg, creg)
circ.initialize(state, indexes[1:])

Transpose(rowReg, columnReg, circ)

circ.h(ancilaReg[0])
Increment(1, circ.qubits, circ)
circ.h(ancilaReg[0])

Transpose(rowReg, columnReg, circ)

circ.measure(indexes, indexes)

provider = IBMQ.get_provider(group='open')
localSimulator = Aer.get_backend('qasm_simulator')
backend = provider.get_backend('ibmq_qasm_simulator')

shots = 2**(m+n)*20 # 8190
result = execute(circ, localSimulator, shots=shots).result()
counts = result.get_counts(circ)
counts, m, n

counts2 = {}
for key in counts:
#    counts2[key[1:]] = counts[key]
     if key[m+n] == '1':
         counts2[key[:-1]] = counts[key]

######
columnReg = QuantumRegister(n)
rowReg = QuantumRegister(m)
ancilaReg = QuantumRegister(1)
creg = ClassicalRegister(n+m+1)

circ = QuantumCircuit(ancilaReg, rowReg, columnReg, creg)
circ.initialize(state, indexes[1:])

circ.h(ancilaReg[0])
Increment(1, circ.qubits, circ)
circ.h(ancilaReg[0])

circ.measure(indexes, indexes)

result = execute(circ, localSimulator, shots=shots).result()
counts = result.get_counts(circ)

for key in counts:
#    counts2[key[1:]] = counts[key]
     if key[m+n] == '1':
         counts2[key[:-1]] = counts[key]
######
            
exportedImg = exportQuantumImage(counts2, shots, 2**m, 2**n)
exportedImg

In [56]:
exportedImg.show()